In [7]:
import os, math, random
import numpy as np
import pandas as pd
import nibabel as nib
from scipy.ndimage import rotate, shift, zoom
from skimage.util import random_noise
from pathlib import Path

In [5]:
# 入力CSV、出力CSV、および拡張画像の保存フォルダのパスを指定
input_csv = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train_augmentation/axial/labels_axial.csv"  # 適宜ファイルパスを変更してください
output_csv = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train_augmentation/axial/augumentation_labels_axial.csv"  # 出力CSVのパス
augmentation_folder = "/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train_augmentation/axial/augmented_images"  # 拡張画像の保存フォルダ

In [ ]:
"""
Axial スライス用データ拡張
  • 0/1 ラベル不均衡を補うため positive(1) を増やす
  • 拡張画像は NIfTI(.nii) で保存し、CSV を更新
"""
# ----------------------- 入出力パス -----------------------
input_csv  = Path("/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train_augmentation/axial/labels_axial.csv")
output_csv = Path("/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train_augmentation/axial/augumentation_labels_axial.csv")
aug_dir    = Path("/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train_augmentation/axial/augmented_images")
aug_dir.mkdir(parents=True, exist_ok=True)

# ------------------- 不均衡補正パラメータ ------------------
target_ratio     = 1.0    # 1 クラス : 0 クラス ≈ 1:1 になるまで増やす
max_aug_per_img  = 10     # 1 枚あたりの最大拡張枚数

# ------------------- 拡張パラメータ ----------------------
max_rot      = 40         # deg
max_shift    = 0.15       # ±15% 平行移動
scale_range  = (0.9, 1.1) # 拡大縮小
noise_var    = 0.01       # ガウスノイズ σ²

# ----------------------- 関数群 --------------------------
def pad_for_rotation(img, angle_deg):
    """回転後の欠損を防ぐゼロパディング"""
    h, w = img.shape[:2]
    rad  = math.radians(abs(angle_deg))
    new_h = int(np.ceil(w*math.sin(rad) + h*math.cos(rad)))
    new_w = int(np.ceil(h*math.sin(rad) + w*math.cos(rad)))
    padded = np.zeros((new_h, new_w)+img.shape[2:], img.dtype)
    dh, dw = (new_h-h)//2, (new_w-w)//2
    padded[dh:dh+h, dw:dw+w] = img
    return padded, dh, dw

def random_affine(img):
    """回転 / 平行移動 / 拡大縮小 のいずれか + 左右反転 + ノイズ"""
    h, w = img.shape[:2]
    r = random.random()

    if r < 0.4:                               # ----- 回転 -----
        ang = random.uniform(-max_rot, max_rot)
        pad, dh, dw = pad_for_rotation(img, ang)
        out = rotate(pad, ang, axes=(0,1), reshape=False, order=1, mode='nearest')
        out = out[dh:dh+h, dw:dw+w]

    elif r < 0.7:                             # ----- 平行移動 -----
        sh = random.uniform(-max_shift, max_shift) * h
        sw = random.uniform(-max_shift, max_shift) * w
        out = shift(img, shift=(sh, sw, 0) if img.ndim==3 else (sh, sw), mode='nearest')

    else:                                     # ----- 拡大縮小 -----
        sc = random.uniform(*scale_range)
        z  = zoom(img, (sc, sc, 1) if img.ndim==3 else (sc, sc), order=1)
        zh, zw = z.shape[:2]
        if zh >= h and zw >= w:               # クロップ
            out = z[(zh-h)//2:(zh+h)//2, (zw-w)//2:(zw+w)//2]
        else:                                 # パディング
            out = np.zeros_like(img)
            dh, dw = (h-zh)//2, (w-zw)//2
            out[dh:dh+zh, dw:dw+zw] = z

    # ----- 左右反転（axial 専用） -----
    if random.random() < 0.5:
        out = np.fliplr(out)

    # ----- ガウスノイズ -----
    out = random_noise(out, mode='gaussian', var=noise_var)
    return (out*255).astype(np.uint8)

# ----------------------- メイン -------------------------
df = pd.read_csv(input_csv)

# ★ 列名が異なる場合はここを書き換え
pos_df = df[df['Fracture_Label'] == 1]
neg_df = df[df['Fracture_Label'] == 0]

print(f"Positive:{len(pos_df)}  Negative:{len(neg_df)}")

desired_pos = int(max(len(neg_df)*target_ratio, len(pos_df)))
needed_aug  = desired_pos - len(pos_df)
print(f"追加が必要な positive 枚数: {needed_aug}")

new_rows = []
if needed_aug > 0:
    made = 0
    # positive をシャッフルして必要数だけループ
    for _, row in pos_df.sample(frac=1, replace=True, random_state=42).iterrows():
        if made >= needed_aug:
            break

        img_nii = nib.load(row['FullPath'])
        img_arr = img_nii.get_fdata()
        remain  = needed_aug - made
        n_aug   = min(max_aug_per_img, remain)

        for k in range(n_aug):
            aug = random_affine(img_arr)
            aug_name = f"{Path(row['FullPath']).stem}_aug{made+k+1}.nii"
            aug_path = aug_dir / aug_name
            nib.save(nib.Nifti1Image(aug, affine=img_nii.affine), aug_path)

            new_row = row.copy()
            new_row['FullPath'] = str(aug_path)
            new_rows.append(new_row)
        made += n_aug

    print(f"実際に追加した枚数: {made}")

# -------------------- CSV 更新 -------------------------
aug_df = pd.DataFrame(new_rows)
updated_df = pd.concat([df, aug_df], ignore_index=True)
updated_df.to_csv(output_csv, index=False)

print(f"=== 完了 ===")
print(f"  元データ: {df.shape[0]}")
print(f"  追加データ: {aug_df.shape[0]}")
print(f"  合計: {updated_df.shape[0]}")
print(f"拡張画像 → {aug_dir}")
print(f"更新 CSV → {output_csv}")


Positive:4595  Negative:33142
追加が必要な positive 枚数: 28547
実際に追加した枚数: 28547
=== 完了 ===
  元データ: 37737
  追加データ: 28547
  合計: 66284
拡張画像 → /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train_augmentation/axial/augmented_images
更新 CSV → /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae/Sakaguchi_file/slice_train_augmentation/axial/augumentation_labels_axial.csv
